In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sales-forecasting/train.csv


In [20]:
df = pd.read_csv('/kaggle/input/sales-forecasting/train.csv')
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


In [21]:
cols = ['Row ID','Order ID','Ship Date','Ship Mode','Customer ID','Customer Name','Segment','Country','City','State','Postal Code','Region','Product ID','Category','Product Name']
df.drop(cols,axis = 1,inplace = True)

In [22]:
df['Order Date'] = pd.to_datetime(df['Order Date'],format = "%d/%m/%Y")
print(df.dtypes)
df = df.sort_values('Order Date')
df.set_index('Order Date',inplace = True)

Order Date      datetime64[ns]
Sub-Category            object
Sales                  float64
dtype: object


In [23]:
all_categories = df['Sub-Category'].unique()
all_categories
df.dtypes

Sub-Category     object
Sales           float64
dtype: object

In [24]:
df.reset_index(inplace = True)
df

,Order Date,Sub-Category,Sales
0,2015-01-03,Paper,16.448
1,2015-01-04,Binders,3.540
2,2015-01-04,Storage,272.736
3,2015-01-04,Labels,11.784
4,2015-01-05,Art,19.536
...,...,...,...
9795,2018-12-30,Binders,20.720
9796,2018-12-30,Bookcases,323.136
9797,2018-12-30,Appliances,209.300
9798,2018-12-30,Binders,52.776


In [28]:
agg = df.groupby(['Order Date', 'Sub-Category']).sum().reset_index()
pivot_sales = agg.pivot(index='Order Date', columns='Sub-Category', values='Sales')
pivot_sales = pivot_sales.fillna(0)
all_categories = df['Sub-Category'].unique()
pivot_sales = pivot_sales.reindex(columns=all_categories, fill_value=0)

In [26]:
pivot_sales

Sub-Category,Paper,Binders,Storage,Labels,Art,Phones,Chairs,Fasteners,Furnishings,Accessories,Envelopes,Bookcases,Appliances,Tables,Supplies,Machines,Copiers
Order Date,,,,,,,,,,,,,,,,,
2015-01-03,16.448,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2015-01-04,0.000,3.540,272.736,11.784,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2015-01-05,0.000,0.000,0.000,0.000,19.536,0.000,0.000,0.000,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2015-01-06,25.980,609.980,0.000,0.000,18.260,1147.940,2573.820,31.120,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2015-01-07,0.000,10.430,0.000,0.000,0.000,0.000,0.000,0.000,76.728,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-26,60.782,3.132,750.680,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2018-12-27,0.000,0.000,0.000,0.000,13.248,164.388,0.000,0.000,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2018-12-28,4.280,62.032,908.874,72.450,29.648,14.850,454.076,13.960,18.328,0.0,0.0,78.8528,0.0,0.0,0.0,0.0,0.0


In [30]:
pivot_sales.index = pd.to_datetime(pivot_sales.index)
date_range = pd.date_range(start='2015-01-03', end='2018-12-30', freq='D')
date_range = pd.to_datetime(date_range)
data_reindexed = pivot_sales.reindex(date_range)
data_filled_forward = data_reindexed.ffill()
data_filled_forward

Sub-Category,Paper,Binders,Storage,Labels,Art,Phones,Chairs,Fasteners,Furnishings,Accessories,Envelopes,Bookcases,Appliances,Tables,Supplies,Machines,Copiers
2015-01-03,16.448,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2015-01-04,0.000,3.540,272.736,11.784,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2015-01-05,0.000,0.000,0.000,0.000,19.536,0.000,0.000,0.000,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2015-01-06,25.980,609.980,0.000,0.000,18.260,1147.940,2573.820,31.120,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2015-01-07,0.000,10.430,0.000,0.000,0.000,0.000,0.000,0.000,76.728,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-26,60.782,3.132,750.680,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2018-12-27,0.000,0.000,0.000,0.000,13.248,164.388,0.000,0.000,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
2018-12-28,4.280,62.032,908.874,72.450,29.648,14.850,454.076,13.960,18.328,0.0,0.0,78.8528,0.0,0.0,0.0,0.0,0.0
2018-12-29,209.700,12.530,0.000,0.000,34.580,302.376,2161.138,25.630,169.580,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0


Here I have divided the sales for each day based on their particular subcategories. So for prediction we will take input from the user which subcategory sales he wants to predict and we will train the model based on that.

In [11]:
cat = input()

 Paper


In [12]:
data = pd.DataFrame(pivot_sales[cat])
data.info

<bound method DataFrame.info of               Paper
Order Date         
2015-01-03   16.448
2015-01-04    0.000
2015-01-05    0.000
2015-01-06   25.980
2015-01-07    0.000
...             ...
2018-12-26   60.782
2018-12-27    0.000
2018-12-28    4.280
2018-12-29  209.700
2018-12-30    0.000

[1230 rows x 1 columns]>

In [13]:
data.reset_index(inplace = True)
data

,Order Date,Paper
0,2015-01-03,16.448
1,2015-01-04,0.000
2,2015-01-05,0.000
3,2015-01-06,25.980
4,2015-01-07,0.000
...,...,...
1225,2018-12-26,60.782
1226,2018-12-27,0.000
1227,2018-12-28,4.280
1228,2018-12-29,209.700


In [14]:
data.columns = ['ds','y']

In [15]:
!pip install neuralprophet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 16.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.0 MB/s eta 0:00:0000:01
  Attempting uninstall: holidays
    Found existing installation: holidays 0.24
    Uninstalling holidays-0.24:
      Successfully uninstalled holidays-0.24


In [16]:
from neuralprophet import NeuralProphet
my_model = NeuralProphet()
my_model.fit(data,freq = 'D',epochs = 100)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /opt/conda/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 85.122% of the data.
WARNING - (py.warnings._showwarnmsg) - /opt/conda/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /opt/conda/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is depre

Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (39) is too small than the required number                     for the learning rate finder (228). The results might not be optimal.
2024-07-05 05:07:50.703649: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 05:07:50.703802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 05:07:50.886999: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Finding best initial lr:   0%|          | 0/228 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

WARNING - (py.warnings._showwarnmsg) - /opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 14. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.



Training: |          | 0/? [00:00<?, ?it/s]

,train_loss,reg_loss,MAE,RMSE,Loss,RegLoss,epoch
0,0.500882,0.0,234.307297,298.661163,0.500770,0.0,0
1,0.354914,0.0,178.969879,230.525558,0.355173,0.0,1
2,0.241103,0.0,134.456711,177.849380,0.240815,0.0,2
3,0.171105,0.0,105.326889,143.574707,0.171246,0.0,3
4,0.130179,0.0,85.846855,120.582054,0.129042,0.0,4
...,...,...,...,...,...,...,...
95,0.091947,0.0,61.303730,103.924156,0.091891,0.0,95
96,0.091925,0.0,61.411194,103.738686,0.092719,0.0,96
97,0.091950,0.0,60.737179,102.824341,0.090881,0.0,97
98,0.091934,0.0,61.247051,105.718124,0.092402,0.0,98


In [17]:
future_dates = my_model.make_future_dataframe(data,periods=36)
future = my_model.predict(future_dates)
future.head()

WARNING - (py.warnings._showwarnmsg) - /opt/conda/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 85.122% of the data.
WARNING - (py.warnings._showwarnmsg) - /opt/conda/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /opt/conda/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.t

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


,ds,y,yhat1,trend,season_yearly,season_weekly
0,2018-12-31,None,68.870071,64.353500,-3.994206,8.510779
1,2019-01-01,None,69.748070,64.429222,-5.185283,10.504129
2,2019-01-02,None,45.001236,64.504929,-6.314968,-13.188727
3,2019-01-03,None,40.566238,64.580650,-7.388093,-16.626322
4,2019-01-04,None,52.363796,64.656364,-8.402987,-3.889582
